In [1]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pybedtools
import scipy.stats as stats
import seaborn as sns
import joblib
import pathlib
from statsmodels.stats.multitest import multipletests

## Parameters

In [2]:
# relavent score cutoff
rs_cutoff = 0.3
min_dmr_to_test = 1000 # on either side

# motif enrichment
or_cutoff = 1.6
neg_lgp_cutoff = 10
mask_quantile_to_max = 0.8

## Load Data

### Motif gene

In [3]:
motif_gene_anno = pd.read_csv(
    '/home/hanliu/project/mouse_rostral_brain/study/MotifClustering/JASPAR2020_CORE_vertebrates_non-redundant.mouse_genes.with_motif_group.199.csv', 
    index_col=0
)
motif_gene_anno.head()

,motif_name,motif_genes,gene_ids,gene_names,motif_group
motif_uid,,,,,
MA0006.1,Ahr::Arnt,"Ahr,Arnt","ENSMUSG00000019256.17,ENSMUSG00000015522.18","Ahr,Arnt",MotifGroup178
MA0854.1,Alx1,Alx1,ENSMUSG00000036602.14,Alx1,MotifGroup3
MA0634.1,ALX3,ALX3,ENSMUSG00000014603.3,Alx3,MotifGroup3
MA0853.1,Alx4,Alx4,ENSMUSG00000040310.12,Alx4,MotifGroup3
MA0007.3,Ar,Ar,ENSMUSG00000046532.8,Ar,MotifGroup32


### Node Data

In [4]:
adata = anndata.read_h5ad('PairwiseDMR.h5ad')

In [5]:
use_dmr = adata.var_names[((adata.X != 0).sum(axis=0) != 0).A1]

### DMR Bed and Rate

In [7]:
with pd.HDFStore('/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_DissectionRegion/DMRInfo.h5') as hdf:
    dmr_bed_df = hdf['Bed'].loc[use_dmr].copy()
    dmr_rate = hdf['Rate'].loc[use_dmr].copy()

### DMR annot

In [9]:
dmr_annot = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_DissectionRegion/MotifScan.h5ad'
)
# mask small motif scores
motif_cutoff = pd.Series(dmr_annot.X.max(axis=0).todense().A1 * mask_quantile_to_max, index=dmr_annot.var_names)

dmr_annot = dmr_annot[use_dmr, :].copy()
dmr_annot

AnnData object with n_obs × n_vars = 343180 × 719 
    obs: 'chrom', 'start', 'end'

## Refilter scores

In [10]:
# only keep value larger than the cutoff for each motif
dmr_annot.X = dmr_annot.X.multiply(
    (dmr_annot.X >
     motif_cutoff[dmr_annot.var_names].values[None, :]).astype(int)).tocsr()

## Prepare test input

In [11]:
def prepare_table(pair_id):
    # get pair dmr
    a, b = adata.obs.loc[pair_id]
    this_dmrs = adata.var_names[adata.var_vector(pair_id).astype(bool)]
    this_dmr_rate = dmr_rate.loc[this_dmrs, [a, b]]
    a_hypo = this_dmr_rate.index[this_dmr_rate[a] < this_dmr_rate[b]]
    b_hypo = this_dmr_rate.index[this_dmr_rate[a] > this_dmr_rate[b]]
    
    # get dmr motif hits annotation
    left_dmr_annot = dmr_annot[a_hypo, :]
    right_dmr_annot = dmr_annot[b_hypo, :]

    # if DMR is not enough, skip and return empty record
    if (a_hypo.size < min_dmr_to_test) or (b_hypo.size < min_dmr_to_test):
        empty_record = pd.DataFrame([],
                                    columns=[
                                        'oddsratio', 'p_value', 'adj_p',
                                        '-lgp', 'left_hit', 'left_no_hit',
                                        'right_hit', 'right_no_hit',
                                        'left_hit_rate', 'right_hit_rate',
                                        'Node'
                                    ])
        return empty_record

    # get table
    motif_ids = dmr_annot.var_names
    # calculate motif occurence, not considering hits here
    left = (left_dmr_annot[:, motif_ids].X > 0).sum(axis=0)
    left_total = left_dmr_annot.shape[0]

    right = (right_dmr_annot.X > 0).sum(axis=0)
    right_total = right_dmr_annot.shape[0]

    tables = {}
    for motif, _left, _right in zip(motif_ids, left.A1, right.A1):
        table = [[_left, left_total - _left], [_right, right_total - _right]]
        tables[motif] = table
    return tables

In [12]:
table_records = {}
for pair_id in adata.obs_names:
    print(pair_id)
    a, b = adata.obs.loc[pair_id]
    data = prepare_table(pair_id)
    table_records[(a, b)] = data

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [13]:
joblib.dump(table_records, 'PairMotifCountTables.lib')

['PairMotifCountTables.lib']

## Test

In [14]:
def test_one_pair(a, b, tables):
    # do test
    results = {}    
    for motif, t in tables.items():
        odds, p = stats.fisher_exact(t, alternative='two-sided')    
        results[motif] = {'oddsratio': odds, 'p_value': p}
    motif_enrich_df = pd.DataFrame(results).T

    # p value correction
    _, p, _, _ = multipletests(motif_enrich_df['p_value'], method='fdr_bh')
    motif_enrich_df['adj_p'] = p
    motif_enrich_df['-lgp'] = -np.log10(motif_enrich_df['adj_p']).replace(
        -np.inf, -300)

    # assemble final results
    records = {}
    for motif, t in tables.items():
        tp, tn = t[0]
        fp, fn = t[1]
        tp_rate = tp / (tp + tn)
        fp_rate = fp / (fp + fn)
        records[motif] = dict(left_hit=tp,
                              left_no_hit=tn,
                              right_hit=fp,
                              right_no_hit=fn,
                              left_hit_rate=tp_rate,
                              right_hit_rate=fp_rate)
    counts = pd.DataFrame(records).T
    motif_enrich_df = pd.concat([motif_enrich_df, counts], axis=1, sort=True)
    motif_enrich_df['ClusterA'] = a
    motif_enrich_df['ClusterB'] = b
    
    # apply a minimum filter
    motif_enrich_df = motif_enrich_df[motif_enrich_df['-lgp'] > 2]

    return motif_enrich_df

In [15]:
table_records = joblib.load('PairMotifCountTables.lib')

In [16]:
temp_dir = 'TEMP'
pathlib.Path(temp_dir).mkdir(exist_ok=True)

In [ ]:
with ProcessPoolExecutor(40) as executor:
    futures = {}
    for (a, b), tables in table_records.items():
        if not isinstance(tables, dict):
            continue
        output_path = f'{temp_dir}/{a}-{b}.msg'
        if pathlib.Path(output_path).exists():
            continue
        
        future = executor.submit(test_one_pair, a, b, tables)
        futures[future] = (a, b, output_path)
        
    for future in as_completed(futures):
        a, b, output_path = futures[future]
        data  = future.result()
        data.to_msgpack(output_path)
        print(a, b, data.shape[0], sep='\t')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  app.launch_new_instance()


IT-L4+MOp-2	IT-L4+SSp-3	66
IT-L4+MOp-2	IT-L4+SSp-4	67
IT-L4+MOp-2	IT-L4+MOp-1	67
IT-L4+MOp-2	IT-L4+SSp-5	110
IT-L4+MOp-2	IT-L4+MOs-3	57
IT-L4+MOp-2	IT-L4+MOs-1	98


/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


IT-L4+MOp-2	IT-L23+SSp-2	309
IT-L4+MOp-2	IT-L23+SSp-3	314


/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


...

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


IT-L4+SSp-1	IT-L23+MOs-2	380
IT-L4+SSp-1	IT-L23+MOs-3	362
IT-L4+SSp-1	IT-L23+MOs-1	385
IT-L4+SSp-1	IT-L23+ACA-3	354
IT-L4+SSp-1	IT-L23+PFC-2	414
IT-L4+SSp-1	IT-L23+ACA-2	371
IT-L4+SSp-1	IT-L23+PFC-1	426
IT-L4+SSp-1	IT-L23+ACA-1	350
IT-L4+SSp-1	IT-L23+AI	404
IT-L4+SSp-1	IT-L23+ORB	250
IT-L4+SSp-1	IT-L5+SSp-2	161
IT-L4+SSp-1	IT-L5+MOp-2	253
IT-L4+SSp-1	IT-L6+SSs-2	161
IT-L4+SSp-1	IT-L5+SSp-3	174
IT-L4+SSp-1	IT-L5+SSp-4	154
IT-L4+SSp-1	IT-L5+SSs-1	256
IT-L4+SSp-1	IT-L5+SSs-2	201
IT-L4+SSp-1	IT-L5+MOp-3	245
IT-L4+SSp-1	IT-L6+SSp-2	212
IT-L4+SSp-1	IT-L6+SSp-4	169
IT-L4+SSp-1	IT-L6+SSp-3	202
IT-L4+SSp-1	IT-L6+SSs-1	110
IT-L4+SSp-1	IT-L6+SSp-1	195
IT-L4+SSp-1	IT-L5+MOs-2	321
IT-L4+SSp-1	IT-L5+SSp-5	166
IT-L4+SSp-1	IT-L6+SSp-5	132
IT-L4+SSp-1	IT-L5+SSp-1	173
IT-L4+SSp-1	IT-L5+MOs-3	288
IT-L4+SSp-1	IT-L6+ACA-3	239
IT-L4+SSp-1	IT-L6+ACA-2	232
IT-L4+SSp-1	IT-L6+MOs-2	272
IT-L4+SSp-1	IT-L6+ACA-1	243
IT-L4+SSp-1	IT-L6+MOs-3	257
IT-L4+SSp-1	IT-L6+MOp-2	258
IT-L4+SSp-1	IT-L5+MOp-4	209
IT-L4+SSp-1	IT-

IT-L4+MOs-3	IT-L5+ACA-2	256
IT-L4+MOs-3	IT-L6+MOp-2	226
IT-L4+MOs-3	IT-L5+ACA-1	225
IT-L4+MOs-3	IT-L6+MOs-2	227
IT-L4+MOs-3	IT-L5+PFC-1	381
IT-L4+MOs-3	IT-L6+SSp-5	156
IT-L4+MOs-3	IT-L5+ORB	266
IT-L4+MOs-3	IT-L6+PFC-1	222
IT-L4+MOs-3	IT-L6+MOs-3	244
IT-L4+MOs-3	IT-L6+MOp-4	234
IT-L4+MOs-3	IT-L5+AI	356
IT-L4+MOs-3	IT-L6+AI	223
IT-L4+MOs-3	IT-L5+PFC-2	355
IT-L4+MOp-3	IT-L23+SSp-1	311
IT-L4+MOp-3	IT-L23+SSs-1	331
IT-L4+MOp-3	IT-L23+SSp-2	274
IT-L4+MOp-3	IT-L23+SSs-2	322
IT-L4+MOp-3	IT-L23+SSp-3	299
IT-L4+MOp-3	IT-L23+SSp-4	292
IT-L4+MOp-3	IT-L23+MOp-3	306
IT-L4+MOp-3	IT-L23+SSp-5	297
IT-L4+MOp-3	IT-L23+MOp-2	305
IT-L4+MOp-3	IT-L23+MOp-4	297
IT-L4+MOp-3	IT-L23+MOs-2	329
IT-L4+MOp-3	IT-L23+MOp-1	312
IT-L4+MOp-3	IT-L23+MOs-1	330
IT-L4+MOp-3	IT-L23+MOs-3	314
IT-L4+MOp-3	IT-L23+AI	345
IT-L4+MOp-3	IT-L23+ACA-3	329
IT-L4+MOp-3	IT-L23+ACA-2	327
IT-L4+MOp-3	IT-L23+ORB	228
IT-L4+MOp-3	IT-L23+ACA-1	338
IT-L4+MOp-3	IT-L23+PFC-2	367
IT-L4+MOp-3	IT-L23+PFC-1	373
IT-L4+MOp-3	IT-L5+MOs-2	306
IT-L4+MOp-4	

IT-L23+SSp-1	IT-L5+SSs-2	348
IT-L23+SSp-1	IT-L5+SSp-4	307
IT-L23+SSp-1	IT-L5+MOp-3	326
IT-L23+SSp-1	IT-L6+PFC-1	241
IT-L23+SSp-1	IT-L5+SSs-1	352
IT-L23+SSp-1	IT-L5+MOs-2	310
IT-L23+SSp-1	IT-L5+MOs-3	303
IT-L23+SSp-1	IT-L5+SSp-5	290
IT-L23+SSp-1	IT-L5+SSp-1	328
IT-L23+SSp-1	IT-L5+MOp-4	294
IT-L23+SSp-1	IT-L5+ACA-3	305
IT-L23+SSs-1	IT-L23+ACA-3	197
IT-L23+SSs-1	IT-L23+ACA-2	206
IT-L23+SSs-1	IT-L23+ORB	163
IT-L23+SSp-1	IT-L5+PFC-1	343
IT-L23+SSp-1	IT-L5+ACA-2	316
IT-L23+SSs-1	IT-L23+PFC-1	261
IT-L23+SSs-1	IT-L23+ACA-1	211
IT-L23+SSp-1	IT-L5+ACA-1	299
IT-L23+SSs-1	IT-L23+PFC-2	232
IT-L23+SSp-1	IT-L5+MOs-1	331
IT-L23+SSp-1	IT-L5+MOp-1	310
IT-L23+SSs-1	IT-L6+SSp-2	340
IT-L23+SSp-1	IT-L5+AI	345
IT-L23+SSs-1	IT-L6+MOs-2	329
IT-L23+SSs-1	IT-L6+MOs-3	309
IT-L23+SSs-1	IT-L6+MOp-3	328
IT-L23+SSs-1	IT-L6+SSp-3	304
IT-L23+SSp-1	IT-L5+PFC-2	319
IT-L23+SSs-1	IT-L6+SSs-2	253
IT-L23+SSs-1	IT-L6+SSp-4	275
IT-L23+SSs-1	IT-L6+SSp-1	262
IT-L23+SSp-1	IT-L5+ORB	314
IT-L23+SSs-1	IT-L6+ACA-2	266
IT-L23+SSs-1	IT

IT-L23+SSp-5	IT-L6+SSs-1	208
IT-L23+SSp-5	IT-L6+SSp-5	262
IT-L23+SSp-5	IT-L6+MOp-4	292
IT-L23+SSp-3	IT-L5+AI	358
IT-L23+SSp-5	IT-L6+ACA-2	263
IT-L23+SSp-5	IT-L6+ACA-1	250
IT-L23+SSp-5	IT-L5+SSp-2	327
IT-L23+SSp-5	IT-L6+PFC-1	239
IT-L23+SSp-3	IT-L5+PFC-2	321
IT-L23+SSp-5	IT-L6+AI	254
IT-L23+SSp-5	IT-L6+PFC-2	233
IT-L23+MOp-4	IT-L23+AI	88
IT-L23+SSp-5	IT-L5+MOp-2	294
IT-L23+SSp-5	IT-L5+SSp-5	250
IT-L23+SSp-5	IT-L5+SSs-1	294
IT-L23+SSp-5	IT-L5+SSp-4	277
IT-L23+MOp-4	IT-L23+ACA-2	122
IT-L23+MOp-4	IT-L23+ACA-3	105
IT-L23+SSp-5	IT-L5+MOp-3	294
IT-L23+SSp-3	IT-L5+ORB	312
IT-L23+SSp-5	IT-L5+MOp-4	265
IT-L23+SSp-5	IT-L5+MOs-2	273
IT-L23+SSp-5	IT-L5+SSp-3	308
IT-L23+MOp-4	IT-L23+ORB	135
IT-L23+MOp-4	IT-L23+ACA-1	178
IT-L23+SSp-5	IT-L5+MOs-3	262
IT-L23+SSp-5	IT-L5+SSp-1	310
IT-L23+MOp-4	IT-L23+PFC-2	209
IT-L23+SSp-5	IT-L5+SSs-2	324
IT-L23+MOp-4	IT-L23+PFC-1	232
IT-L23+SSp-5	IT-L5+ACA-3	283
IT-L23+SSp-5	IT-L5+ACA-1	274
IT-L23+SSp-5	IT-L5+ACA-2	292
IT-L23+SSp-5	IT-L5+MOs-1	302
IT-L23+SSp-5	IT-L5+PF

In [ ]:
total_enrichment = pd.concat([pd.read_msgpack(p) for p in pathlib.Path(temp_dir).glob('*msg')])
total_enrichment.to_msgpack('PairwiseMotifEnrichment.msg')

In [ ]:
total_enrichment.head()